작업 내용
- 1. countvectorizer 활용 cosine유사도 분석
- 2. tf-idf 활용 cosine유사도 분석
- 로직설명
- 독자-독자 : 취향 유사도
- 독자-작가 : 취향 유사도
- 독자 1명이 읽은 글의 전체 태그를 말뭉치로 만듦
- 작가 1명이 작성한 글의 전체 태그를 말뭉치로 만듦
- 말뭉치를 형태소로 분석
- vectorize 및 cosine 유사도 구하기
    - 중복 비제거 (tf-idf 사용)
- 100명 추천

In [82]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from pandas import Panel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [63]:
from Brunch_data import *

In [58]:
import glob
meta = pd.read_json('C:/Users/eunice/dss_workspace/MLproject/data/metadata.json', lines=True)
read_file_lst = glob.glob('C:/Users/eunice/dss_workspace/MLproject/data/read\*')
maga = pd.read_json('C:/Users/eunice/dss_workspace/MLproject/data/magazine.json', lines=True)
us = pd.read_json('C:/Users/eunice/dss_workspace/MLproject/data/users.json', lines=True)

In [59]:
dev = pd.read_csv("C:\\Users\eunice\\dss_workspace\\MLproject\\data\\predict\\dev.users", header=None, names=["readers_id"])

In [60]:
dev

,readers_id
0,#d6866a498157771069fdf15361cb012b
1,#f963fb8c5d9d14d503fc4e80bd8617b4
2,#87a6479c91e4276374378f1d28eb307c
3,#677e984e245b344f61dc5d3cc1f352c8
4,#519f45eb14e4807e8714fb7e835463eb
...,...
2995,#c8bdf5ee13c87360dd14ffea7f63483f
2996,#e8dbed7a4331fb9332d6e9fdf338e67d
2997,#3322441e6904f096c8a86ea5f263bf2a
2998,#8a990bbf90490dfb2a4c4ef8deffc136


In [64]:
a = Data(meta, read_file_lst, maga, us)
me_df, r_df, mag_df, us_df = a.data_preprocessing(dev)

In [65]:
meta_df = me_df.copy()
read_df = r_df.copy()

# 독자-독자 유사도 구하기
- 독자별 주로 읽은 글의 키워드 정리하기
- 1. train2 데이터와 read데이터의 독자 정보 merge
- 2. train2 데이터에서 독자 id기준 groupby/ keyword list sum
- 3. 키워드 중복 제거
- 4. 키워드 형태소 분석

## Tfidf vectorizer 구성

In [3]:
### author_morphs_final_1109.csv 파일 불러오기 (작가 형태소분석 완료 파일)

In [15]:
train_key_sum = pd.read_csv('./readers_morphs_final_1109.csv', encoding='utf-8', index_col=[0]).reset_index(drop=True)

In [16]:
train_key_sum['morphs_list'] = train_key_sum['morphs_list'].apply(lambda x: str(x).replace('[', '').replace(']','').replace("'", ''))

In [17]:
train_key_sum

,readers_id,keyword_list,morphs_list
0,#000474bba0c00c70e12ac7cfc3d04553,"['패스트파이브', '스타트업', '공유오피스', '마케팅', '게임', '홍보',...","패스트, 파이브, 스타트, 업, 공유, 오피스, 마케팅, 게임, 홍보, 마케팅, 게..."
1,#000c82417339e173dbed1e413e9c15e0,"['브런치', '시사회', '브런치무비패스', '브런치', '시사회', '브런치무비...","브런, 치, 시사회, 브런, 하, 지, 무비, 패스, 브런, 치, 시사회, 브런, ..."
2,#000c9b7827aba665e4c957f4223629fc,"['고양이', '길고양이', '반려동물', '시어머니', '독일', '국제결혼', ...","고양이, 길, 고양이, 반려, 동물, 시어머니, 독일, 국제, 결혼, 직장인, 처세..."
3,#000fe61478d384d09f3bcdd0c2f5227d,"['동기', '재미', '방법', '동기', '재미', '방법', '동기', '재미...","동기, 재미, 방법, 동기, 재미, 방법, 동기, 재미, 방법, 동기, 재미, 방법..."
4,#0013c35c789f0936654ba06389ea75cb,"['육아', '결혼', '시어머니', '육아', '결혼', '시어머니', '며느리'...","육아, 결혼, 시어머니, 육아, 결혼, 시어머니, 며느리, 명절, 시댁, 육아, 결..."
...,...,...,...
28302,#fffb1351938a67654fa921cb47750e72,"['블록체인', '블로그', '스팀잇', '출판', '독립출판', '판매', '검색...","블록, 체인, 블, ㄹ, 로그, 스팀, 잇, 출판, 독립, 출판, 판매, 검색, 광..."
28303,#fffca5f83f8b7145b3102aec2e0934dc,"['사랑', '이별', '에세이', '사랑', '이별', '에세이', '간호사', ...","사랑, 이별, 에세이, 사랑, 이별, 에세이, 간호사, 응급실, 간호, 금, 수저,..."
28304,#fffcc9c67c1dfb2543eccdde30c19d6f,"['직장생활', '직장', '무기력', '직장생활', '직장', '무기력', '회사...","직장, 생활, 직장, 무기력, 직장, 생활, 직장, 무기력, 회사, 호의, 배려, ..."
28305,#fffd0b8737addae9f6e4da8bf04daddd,"['카메라', '취미', '아빠', '운동', '뱃살', '다이어트', '브런치',...","카메라, 취미, 아빠, 운동, 뱃살, 다이어트, 브런, 치, 시사회, 브런, 하, ..."


In [18]:
# instantiation
tfidf_vectorizer = TfidfVectorizer()

In [19]:
# fit_transform
tqdm.pandas()
tf_vec = tfidf_vectorizer.fit_transform(train_key_sum['morphs_list'])
print(tf_vec)

C:\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


  (0, 1135)	0.08285222367000382
  (0, 5449)	0.07291276339783251
  (0, 4850)	0.07957082062944239
  (0, 4400)	0.07882852741846014
  (0, 4803)	0.08939226926794337
  (0, 3893)	0.06377217612577969
  (0, 3069)	0.13646711538708156
  (0, 2245)	0.10790548789723724
  (0, 154)	0.08135502541224288
  (0, 4679)	0.11667174791393894
  (0, 139)	0.15095539787987536
  (0, 1280)	0.1384420006607719
  (0, 4010)	0.10790548789723724
  (0, 2046)	0.10690332894377916
  (0, 5028)	0.05931275490774956
  (0, 2849)	0.11452753683104419
  (0, 4726)	0.162190065618087
  (0, 1616)	0.0697947353201308
  (0, 5124)	0.03585519337407023
  (0, 1708)	0.0361413791452655
  (0, 2969)	0.0361468246164115
  (0, 2224)	0.07041398135235526
  (0, 5022)	0.04155368336440065
  (0, 829)	0.08680274552587032
  (0, 2885)	0.08697295579650595
  :	:
  (28306, 5450)	0.07477090079870086
  (28306, 3080)	0.07490796744579092
  (28306, 3962)	0.07904990778930258
  (28306, 3853)	0.09376840876738593
  (28306, 3954)	0.16746871336935107
  (28306, 675)	0.060669

In [14]:
# cosine유사도 분석을 위해 행렬을 array로 변경
tf_vec = tf_vec.toarray()

---
여기까지 하면 cosine_similarity에 넣을 reader data 준비완료

## cosine 유사도 구하기

In [20]:
# 독자 * 독자 유사도 구하기

reader_keyword_sim = cosine_similarity(tf_vec, tf_vec)
print(reader_keyword_sim.shape)
print(reader_keyword_sim[:2])

(28307, 28307)
[[1.         0.03345728 0.01835777 ... 0.03328841 0.06867242 0.00963103]
 [0.03345728 1.         0.         ... 0.         0.0786649  0.01514257]]


In [21]:
reader_keyword_sim

array([[1.        , 0.03345728, 0.01835777, ..., 0.03328841, 0.06867242,
        0.00963103],
       [0.03345728, 1.        , 0.        , ..., 0.        , 0.0786649 ,
        0.01514257],
       [0.01835777, 0.        , 1.        , ..., 0.02440207, 0.08460645,
        0.07848783],
       ...,
       [0.03328841, 0.        , 0.02440207, ..., 1.        , 0.21638987,
        0.02761123],
       [0.06867242, 0.0786649 , 0.08460645, ..., 0.21638987, 1.        ,
        0.12628587],
       [0.00963103, 0.01514257, 0.07848783, ..., 0.02761123, 0.12628587,
        1.        ]])

In [44]:
# 코사인 유사도 기준으로 sorting하기 (가장 유사한 독자 순으로 나열)
reader_keyword_sim_sorted_ind = reader_keyword_sim.argsort()[:,::-1]
print(keyword_sim_sorted_ind[:1])

[[ 561 1037  505 ... 2512 2511    0]]


# 독자-작가 유사도 구하기

## 작가 형태소 분석 파일 불러오기

In [27]:
# 저장파일 불러오기
author_key_sum = pd.read_csv('author_morphs_final_1109_0228.csv', encoding='utf-8', index_col=[0]).reset_index(drop=True)

In [28]:
author_key_sum['morphs_list'] = author_key_sum['morphs_list'].apply(lambda x: str(x).replace('[', '').replace(']','').replace("'", ''))
author_key_sum

,author_id,keyword_list,morphs_list
0,@01026856762,"['부모교육', '학교폭력', '가정', '자립', '행복', '아들러', '부모교...","부모, 교육, 학교, 폭력, 가정, 자리, 행복, 아들러, 부모, 교육, 생각, 교..."
1,@01038273527,"['신용카드', '재테크', '부자', '비전', '자기관리', '꿈', '요리',...","신용, 카드, 재테크, 부자, 비전, 자기관리, 꿈, 요리, 갈비, 요리, 방법, ..."
2,@01archive,"['영화', '나나미', '배우', '영화', '사랑', '영화', '계곡']","영화, 나, 아, 나미, 배우, 영화, 사랑, 영화, 계곡"
3,@02jsh0314,"['영어', '유치원', '학원', '영어유치원', '영어', '교실', '영어유치...","영어, 유치원, 학원, 영어, 유치원, 영어, 교실, 영어, 유치원, 영어, 유치원"
4,@0315hhj,"['제주', '욕심', '공감에세이']","제주, 욕심, 공감, 에, 세, 이"
...,...,...,...
3973,@zxc2111,"['후기', '일상']","후기, 일상"
3974,@zyeun,"['일상', '에세이', '생각', '이별', '에세이', '연애', '연애', '...","일상, 에세이, 생각, 이별, 에세이, 연애, 연애, 이별, 에세이"
3975,@zzing-gu,"['공감에세이', '글쓰기', '시', '글쓰기', '공감에세이', '시', '공감...","공감, 에, 세, 이, 글쓰기, 시, 글쓰기, 공감, 에, 세, 이, 시, 공감, ..."
3976,@zzintta,"['웹툰', '웹소설', '좀비', '웹소설', '웹툰', '좀비']","웹, 툰, 웹, 소설, 좀비, 웹, 소설, 웹, 툰, 좀비"


## Tfidf vectorizer 구성
- 독자-작가의 cosine 유사도를 구하기 위해서는 다시 instantiation하지 않음
- fit_transform이 아닌 transform만 진행함

In [30]:
# 독자-작가 유사도를 구하기 위한 작가 형태도의 vector화 (transform만 진행하면 됨)
tqdm.pandas()
author_tf_vec = tfidf_vectorizer.transform(author_key_sum['morphs_list'])
print(author_tf_vec)

  (0, 5470)	0.14870229146874347
  (0, 5357)	0.15846048795538187
  (0, 5209)	0.281541506266957
  (0, 4024)	0.29694378135622723
  (0, 3113)	0.4302476960596866
  (0, 2945)	0.17331201353265013
  (0, 2895)	0.17537976891617962
  (0, 2480)	0.11353954826782423
  (0, 2140)	0.3582806622369377
  (0, 566)	0.4651162028955309
  (0, 308)	0.30797829955514966
  (0, 192)	0.2949381269699537
  (1, 5677)	0.18720703838151392
  (1, 5470)	0.11291772856102097
  (1, 4749)	0.15575026533858824
  (1, 4733)	0.18461272965785264
  (1, 4726)	0.20687741655985645
  (1, 4505)	0.21490312535922987
  (1, 4477)	0.16187978139622647
  (1, 4474)	0.13284108482234422
  (1, 4158)	0.22791000300781641
  (1, 4149)	0.10322427897733946
  (1, 4053)	0.10914435625409487
  (1, 4014)	0.23872008319407662
  (1, 3992)	0.18757709211705448
  :	:
  (3972, 193)	0.20555014467324026
  (3973, 5641)	0.9211337022461755
  (3973, 3965)	0.3892463263619248
  (3974, 3965)	0.20602518780175208
  (3974, 3858)	0.6747810835756165
  (3974, 3404)	0.456113991709930

C:\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [31]:
# 행렬 array로 변경하기
author_tf_vec = author_tf_vec.toarray()

## 앞에서 구한 독자 키워드, 작가 키워드로 cosine유사도 구하기
- 키워드는 벡터변환 후 toarray()까지 해서 유사도를 구해야 함
- 독자 키워드 리스트 말뭉치 vector화 변수: tf_vec
- 작가 키워드 리스트 말뭉치 vector화 변수: authoor_tf_vec
- instantiation은 독자에서만 해주고, 작가는 transform만 해주어 내적을 진행할 수 있도록 만들어주어야 함

In [32]:
tf_vec.shape

(28307, 5687)

In [33]:
author_tf_vec.shape

(3978, 5687)

In [34]:
# 독자-작가 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity

reader_author_keyword_sim = cosine_similarity(tf_vec, author_tf_vec)
print(reader_author_keyword_sim.shape)
print(reader_author_keyword_sim[:2])

(28307, 3978)
[[0.         0.04724982 0.         ... 0.         0.         0.00613554]
 [0.         0.         0.         ... 0.         0.         0.02317824]]


In [35]:
# 코사인 유사도 결과물
reader_author_keyword_sim

array([[0.        , 0.04724982, 0.        , ..., 0.        , 0.        ,
        0.00613554],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.02317824],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.01742944],
       ...,
       [0.        , 0.03384542, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.0483044 , 0.        , ..., 0.        , 0.        ,
        0.02484379],
       [0.0075408 , 0.01830141, 0.        , ..., 0.        , 0.        ,
        0.08380046]])

In [43]:
# 유사도 행렬들에서 가장 높은 순으로 정렬 시키기
author_keyword_sim_sorted_ind = reader_author_keyword_sim.argsort()[:,::-1]
print(keyword_sim_sorted_ind[:1])

[[ 561 1037  505 ... 2512 2511    0]]


In [39]:
keyword_sim_sorted_ind

array([[ 561, 1037,  505, ..., 2512, 2511,    0],
       [3160, 3958, 2958, ..., 2482, 2480,    0],
       [ 439, 1347, 2892, ..., 2527, 2526,    0],
       ...,
       [3459, 1861,  467, ..., 2524, 2523,    0],
       [2276,  572, 1822, ..., 2512, 2510,    0],
       [3589, 1639, 2745, ..., 1387, 2746, 1988]], dtype=int64)

# 100개 추천 코드

In [153]:
with open("recom_list_28", 'rb') as f:
    pop_recom = pickle.load(f)
pop_recom

[('@brunch_151', datetime.date(2019, 2, 18), 22734.0),
 ('@mothertive_66', datetime.date(2019, 2, 21), 3592.0),
 ('@brunch_152', datetime.date(2019, 2, 27), 2693.0),
 ('@onyouhe_98', datetime.date(2019, 2, 21), 2634.0),
 ('@ohmygod_42', datetime.date(2019, 2, 8), 2568.0),
 ('@fuggyee_108', datetime.date(2019, 2, 11), 2407.0),
 ('@keeuyo_57', datetime.date(2019, 2, 7), 2319.0),
 ('@roysday_314', datetime.date(2019, 2, 8), 2312.0),
 ('@seochogirl_28', datetime.date(2019, 2, 21), 2309.0),
 ('@boot0715_115', datetime.date(2019, 2, 11), 2250.0),
 ('@choyoungduke_157', datetime.date(2019, 2, 18), 2000.0),
 ('@yoriyuri_12', datetime.date(2019, 2, 9), 1933.0),
 ('@drunk-traveler_49', datetime.date(2019, 2, 13), 1920.0),
 ('@kam_65', datetime.date(2019, 2, 18), 1908.0),
 ('@thebluenile86_4', datetime.date(2019, 2, 11), 1899.0),
 ('@dryjshin_255', datetime.date(2019, 2, 13), 1896.0),
 ('@jijuyeo_9', datetime.date(2019, 2, 8), 1860.0),
 ('@syshine7_57', datetime.date(2019, 2, 7), 1854.0),
 ('@tam

In [154]:
pop_recom = pd.DataFrame(pop_recom, columns=['article_id', "dt", "popular_weight"])

In [196]:
# reader_id를 넣고 author_id를 찾아주는 함수
def find_sim_author(df_reader=train_key_sum, df_author=author_key_sum, sorted_ind=author_keyword_sim_sorted_ind, reader_name="name", top_n=10):
    reader_list = df_reader[df_reader['readers_id']==reader_name]
    
    re_article_index = reader_list.index.values
    similar_indexes = sorted_ind[re_article_index, :(top_n)]
    
    similar_indexes = similar_indexes.reshape(-1)
    
    return df_author.iloc[similar_indexes]

# reader_id를 넣고 유사한 reader 찾아주는 함수
def find_sim_reader(df=train_key_sum, sorted_ind=reader_keyword_sim_sorted_ind, reader_name="name", top_n=10):
    reader_list = df[df['readers_id']==reader_name]
    
    article_index = reader_list.index.values
    similar_indexes = sorted_ind[article_index, :(top_n)]
    
    similar_indexes = similar_indexes.reshape(-1)
    

    return df.iloc[similar_indexes]

## 독자-작가 유사성을 통한 작가가 쓴 글 추천

def users_author_recom(func1=find_sim_author, func2=find_sim_reader, readers_id="name", metadata=meta_df, pop_recom=pop_recom, read=read_df): 

    ## 독자-작가 유사성을 통한 작가가 쓴 글 추천
    authors = list(func1(reader_name=readers_id).author_id) 
    recommend_df = pd.DataFrame(columns=["title", "author_id", "display_url"])
    
    for author in authors:
        author_recom = metadata[metadata.author_id == author].sort_values(by="reg_datetime", ascending=False)
        if len(author_recom) >= 5:
            recommend_df = pd.concat([recommend_df, author_recom[["title", "author_id", "display_url", "article_id"]].iloc[:5]])
        else:
            recommend_df = pd.concat([recommend_df, author_recom[["title", "author_id", "display_url", "article_id"]]])
    
    # 독자-작가 추천 후 추천한 글 제외하기 (겹치지 않기 위해)
    metadata = metadata[~metadata['article_id'].isin(list(recommend_df.article_id))]
    
    ## 독자-독자 유상성을 통한 작가가 쓴 글 추천
    readers = list(func2(reader_name=readers_id).readers_id)

    for reader in readers:
        reader_aritcle_ls=list(read[read.readers_id == reader].article_id)
        reader_recom = metadata[metadata['article_id'].isin(reader_aritcle_ls)].sort_values(by="reg_datetime", ascending=False)
        reader_recom = metadata[metadata.author_id == author].sort_values(by="reg_datetime", ascending=False)
        if len(reader_recom) >= 5:
            recommend_df = pd.concat([recommend_df, reader_recom[["title", "author_id", "display_url", "article_id"]].iloc[:5]])
        else:
            recommend_df = pd.concat([recommend_df, reader_recom[["title", "author_id", "display_url", "article_id"]]])
    
    # 독자-독자 추천 후 추천한 글 제외하기 (겹치지 않기 위해)
    metadata = metadata[~metadata['article_id'].isin(list(recommend_df.article_id))]
    
    
     ## 독자-작가/독자-독자 추천 시에 100이 안될 경우
    if len(recommend_df) == 100:
        recommend_df.reset_index(drop=True, inplace=True)
        return recommend_df
    else:
        count = 100-len(recommend_df)
        popular_recom_article_id = pop_recom.sort_values(by=["dt", "popular_weight"], ascending=False).iloc[:count]
        popular_recom = pd.merge(left=popular_recom_article_id, right=metadata, left_on="article_id", right_on="article_id", how="left")
        
        pd.concat([recommend_df, popular_recom[["title", "author_id", "display_url", "article_id"]]])

    recommend_df.reset_index(drop=True, inplace=True)
    return recommend_df

In [197]:
users_author_recom(readers_id='#000c82417339e173dbed1e413e9c15e0', metadata=me_df, pop_recom=pop_recom, read=read_df)

,title,author_id,display_url,article_id
0,존과 지니의 플로리다 스쿠버 다이빙 여행 11,@skumac,https://brunch.co.kr/@skumac/347,@skumac_347
1,존과 지니의 플로리다 스쿠버 다이빙 여행 10,@skumac,https://brunch.co.kr/@skumac/346,@skumac_346
2,존과 지니의 플로리다 스쿠버 다이빙 여행 9,@skumac,https://brunch.co.kr/@skumac/345,@skumac_345
3,존과 지니의 플로리다 스쿠버 다이빙 여행 8,@skumac,https://brunch.co.kr/@skumac/344,@skumac_344
4,존과 지니의 플로리다 스쿠버 다이빙 여행 7,@skumac,https://brunch.co.kr/@skumac/343,@skumac_343
...,...,...,...,...
95,부실공사,@niyang0721,https://brunch.co.kr/@niyang0721/38,@niyang0721_38
96,암살,@niyang0721,https://brunch.co.kr/@niyang0721/44,@niyang0721_44
97,열쇠,@niyang0721,https://brunch.co.kr/@niyang0721/43,@niyang0721_43
98,나비,@niyang0721,https://brunch.co.kr/@niyang0721/42,@niyang0721_42
